## Додаткове завдання з зірочкою

__Для більшого заглиблення в роботу алгоритму, пропонуємо реалізувати алгоритм колабораційної фільтрації з нуля. Для цього ми можемо скористатись нашою домашньою роботою з 3-ого модуля. Якщо ми модифікуємо функцію втрат та розрахунок градієнтів, то зможемо побудувати алгоритм матричної факторизації.__

Щоб виконати це задання візьмемо наш код з третьго модуля та замінемо його на злегка модіфікований код з обов'язкових відеоматеріалів модуля 7.

In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.io import loadmat
from sklearn.metrics import mean_squared_error, mean_absolute_error

from surprise import accuracy, Dataset, Reader, SVD, SVDpp, NMF
from surprise.model_selection import train_test_split, cross_validate, KFold, GridSearchCV

%matplotlib inline

In [2]:
BASE_FOLDER = Path(Path.cwd(), "data")
data = loadmat(Path(BASE_FOLDER, "movies.mat"))
Y, R = data["Y"], data["R"]

In [3]:
def loadMovieList():
    with open(Path(BASE_FOLDER, "movie_ids.txt"),  encoding='ISO-8859-1') as fid:
        movies = fid.readlines()
    movieNames = []
    for movie in movies:
        parts = movie.split()
        movieNames.append(" ".join(parts[1:]).strip())
    return movieNames

In [4]:
names = loadMovieList()

Функція втрат згідно ДЗ: 
$$ J(x^{(1)}, \dots, x^{(n_m)}, w^{(1)}, \dots, w^{(n_u)}) = \frac{1}{2} \sum_{(i,j):r(i,j)=1} \left( \left( w^{(j)} \right)^T x^{(i)} - y^{(i,j)} \right)^2 + \left( \frac{\lambda}{2} \sum_{j=1}^{n_u} \sum_{k=1}^{n} \left( w_k^{(j)} \right)^2  \right) + \left( \frac{\lambda}{2} \sum_{i=1}^{n_m} \sum_{k=1}^n \left(x_k^{(i)} \right)^2 \right) $$

Градієнти згідно ДЗ:
$$ \frac{\partial J}{\partial x_k^{(i)}} = \sum_{j:r(i,j)=1} \left( \left(w^{(j)}\right)^T x^{(i)} - y^{(i,j)} \right) w_k^{(j)} + \lambda x_k^{(i)} $$

$$ \frac{\partial J}{\partial w_k^{(j)}} = \sum_{i:r(i,j)=1} \left( \left(w^{(j)}\right)^T x^{(i)}- y^{(i,j)} \right) x_k^{(j)} + \lambda w_k^{(j)} $$


__Класс з модуля 3 буде мати наступний вигляд:__

In [5]:
class CollaborativeFiltering:
    def __init__(self, lr: float = 0.001, lambda_: float = 1.0, 
                 thr: float = 0.0001, n_epochs: int = 100, n_factors=10):
        self.lr = lr
        self.lambda_ = lambda_
        self.thr = thr
        self.n_epochs = n_epochs
        self.n_factors = n_factors
        
        self.X = np.array([])
        self.W = np.array([])
        
    def cost(self, Y: np.ndarray, R: np.ndarray) -> float:  # функція втрат
        J = (np.sum((self.X @ self.W.T * R - Y)**2) + self.lambda_ * (np.sum(self.X**2) + np.sum(self.W**2))) / 2
        return J        

    def build_predictions(self) -> np.ndarray:
        return self.X @ self.W.T

    def update_xw(self, Y: np.ndarray, R: np.ndarray):  # крок градієнтного спуску
        X_grad = np.zeros_like(self.X)
        W_grad = np.zeros_like(self.W)
        
        for i in range(R.shape[0]):
            idx = np.where(R[i, :] == 1)[0]
            W_temp = self.W[idx, :]
            Y_temp = Y[i, idx]
            X_grad[i, :] = (self.X[i, :] @ W_temp.T - Y_temp) @ W_temp + self.lambda_ * self.X[i, :]
            
        for j in range(R.shape[1]):
            idx = np.where(R[:, j] == 1)[0]
            X_temp = self.X[idx, :]
            Y_temp = Y[idx, j]
            W_grad[j, :] = (X_temp @ self.W[j, :] - Y_temp) @ X_temp + self.lambda_ * self.W[j, :]
    
        self.X -= (X_grad * self.lr)
        self.W -= (W_grad * self.lr)
        
    def fit(self, Y: np.ndarray, R: np.ndarray):
        self.X = np.random.randn(Y.shape[0], self.n_factors)
        self.W = np.random.randn(Y.shape[1], self.n_factors)
        
        last_cost = 1e+40
        count = 0
        for i in range(self.n_epochs):
            count += 1
            self.update_xw(Y, R)
            new_cost = self.cost(Y, R)
            cost_delta = last_cost - new_cost
            if cost_delta < self.thr:
                break
            last_cost = new_cost
        print(f"count: {count}, last_cost: {last_cost}")        

### Тренування моделі

In [6]:
cf = CollaborativeFiltering(lr = 0.001, lambda_= 20, n_epochs=300, n_factors=20)
cf.fit(Y, R)

count: 300, last_cost: 108161.60496151375


__Спробуємо порахувати RMSE та MAE__

In [7]:
predictions = cf.build_predictions()

y_true = (Y * R).reshape(-1,1)
idx = y_true.nonzero()
y_true = y_true[idx]
y_pred = predictions.reshape(-1, 1)[idx]

print(f"Min rating = {np.min(y_pred)}")
print(f"Max rating = {np.max(y_pred)}")

print(f"\nRMSE = {mean_squared_error(y_true, y_pred, squared=False)}")
print(f"MAE = {mean_absolute_error(y_true, y_pred)}")

Min rating = 0.07476462797609855
Max rating = 5.892192650982537

RMSE = 0.908558756859396
MAE = 0.7175031454245554


Показники помилок при таких параметрах майже співпадають з такими при використанні SVD в першій частині. При зменшенні lambda_ значення помилок також зменшуються, але min rating стає від'ємним, а max rating > 6, що якось нервує і може свідчити про перенавчання.

__Спробуємо побудувати рекомендації для якогось юзера.__

In [8]:
user_id = 100 # np.random.choice(range(R.shape[1]))
idx_new = np.where(R[user_id] == 0)[0]  # викинемо фільми, які він прорейтингував 
user_predict = dict(zip(idx_new, predictions[idx_new, user_id]))
user_predict = dict(sorted(user_predict.items(), key=lambda x: x[1], reverse=True))
top_10_idx = list(user_predict.keys())[:10]
display(pd.DataFrame((np.array(names)[top_10_idx])))

,0
0,Titanic (1997)
1,Good Will Hunting (1997)
2,Star Wars (1977)
3,Air Force One (1997)
4,Return of the Jedi (1983)
5,L.A. Confidential (1997)
6,Raiders of the Lost Ark (1981)
7,Schindler's List (1993)
8,Braveheart (1995)
9,"Rock, The (1996)"


__Спробуємо порівняти з SVD__

In [9]:
data_svd = Dataset.load_from_file(Path(BASE_FOLDER, "u.data"), Reader(line_format="user item rating timestamp"))
data_train = data_svd.build_full_trainset()
algo = SVD(n_factors=150, n_epochs=80, lr_all=0.005, reg_all=0.1)
algo.fit(data_train);

In [10]:
data_test = data_train.build_anti_testset()
predictions_svd = algo.test(data_test)

In [11]:
dict_tmp = {int(predic.iid): predic.est for predic in predictions_svd if predic.uid == "100"}
dict_tmp = dict(sorted(dict_tmp.items(), key=lambda x: x[1], reverse=True))
top_10 = list(dict_tmp.keys())[:10]
display(pd.DataFrame((np.array(names)[top_10]))) 

,0
0,Golden Earrings (1947)
1,Everyone Says I Love You (1996)
2,"Haunted World of Edward D. Wood Jr., The (1995)"
3,What's Eating Gilbert Grape (1993)
4,Cinema Paradiso (1988)
5,Jack (1996)
6,Taxi Driver (1976)
7,So I Married an Axe Murderer (1993)
8,Spanking the Monkey (1994)
9,"Maltese Falcon, The (1941)"


Дуже сумно. Рекомендації навіть приблизно не співпадають. Спишемо це вже на різний смак алгоритмів (чи все-таки на низький рівень знань виконавця?). 

__Напевно розумні люди перед порівнянням алгоритмів спочатку порівнюють вихідні данні.__

Данні для юзера з id = 100, SVD

In [12]:
raw_ratings = (np.array(data_svd.raw_ratings)[:, :3])
raw_ratings = raw_ratings.astype(float).astype(int)
raw_ratings = raw_ratings[(raw_ratings[:, 0] == 100)]
raw_ratings = raw_ratings[raw_ratings[:, 1].argsort()]
df = pd.DataFrame({"mov_id": raw_ratings[:, 1], "rating": raw_ratings[:, 2]})
display(df.head())
df.shape

,mov_id,rating
0,258,4
1,266,2
2,268,3
3,269,4
4,270,3


(59, 2)

Данні для юзера з id = 100, MF

In [13]:
raw_user_100 =  Y[:, 100]
idx = raw_user_100.nonzero()[0]
raw_user_100 = raw_user_100[idx]
df = pd.DataFrame({"mov_id": idx, "rating": raw_user_100})
display(df.head())
df.shape

,mov_id,rating
0,0,3
1,6,3
2,23,4
3,49,4
4,108,2


(67, 2)

__Якщо я нічого не наплутав - данні не співпадають. Для порівняння алгоритмів потрібно їх унікуфувати. Але це вже наступного разу.__

### Висновок: начебто працює.